In [1]:
import argparse
import collections

import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader

from retinanet import coco_eval
from retinanet import csv_eval

assert torch.__version__.split('.')[0] == '1'

print('CUDA available: {}'.format(torch.cuda.is_available()))

CUDA available: True


In [11]:
csv_inference = 'data/test_retinanet.csv'
csv_train = 'data/train_retinanet.csv'
csv_classes = 'data/class_retinanet.csv'
model_path = 'csv_retinanet_68.pt'
dataset_inference = CSVDataset(train_file=csv_inference, class_list=csv_classes,
                                   transform=transforms.Compose([Normalizer(), Resizer()]))

sampler = AspectRatioBasedSampler(dataset_inference, batch_size=1, drop_last=False)
dataloader_inference = DataLoader(dataset_inference, num_workers=3, collate_fn=collater, batch_sampler=sampler)


dataset_train = CSVDataset(train_file=csv_train, class_list=csv_classes,
                                   transform=transforms.Compose([Normalizer(), Resizer()]))

sampler = AspectRatioBasedSampler(dataset_train, batch_size=1, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater, batch_sampler=sampler)

use_gpu = True

retinanet = torch.load(model_path)
retinanet = retinanet.cuda()
retinanet.training = False
retinanet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [57]:
import shap
classifier = retinanet.styleClassificationModel
classifier.eval()
retinanet.style_inference = True
from scipy.special import softmax
def predict(feature_vec):
        res = []
        for el in feature_vec:
            partial_res = classifier.out(classifier.non_linear(classifier.linear(torch.tensor(el).cuda()))).detach().cpu().numpy()
            res.append(softmax(partial_res))
        return np.array(res)

In [55]:
predict(np.array([test_data[0]])).shape

(1, 14)
(14,)
(4,)


(1, 4)

In [16]:
train_data = []
test_data = []
for index in range(len(dataset_train)):
    data = dataset_train[index]
    [_,_,_], style, feature_vec = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    train_data.append(feature_vec.detach().cpu().numpy())
styles = []
styles_bis = []
for index in range(len(dataset_inference)):
    data = dataset_inference[index]
    [_,_,_], style, feature_vec = retinanet(data['img'].permute(2, 0, 1).cuda().float().unsqueeze(dim=0))
    styles.append(np.squeeze(style.detach().cpu().numpy()))
    test_data.append(feature_vec.detach().cpu().numpy())
    pred = predict(test_data[index])
    styles_bis.append(np.squeeze(pred))
train_data = np.array(train_data)
test_data = np.array(test_data)
print(test_data)

Using 196 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  0%|          | 1/219 [00:00<00:25,  8.41it/s]

[[[ 4.575468    0.83224285  1.2297149  ...  0.3909327   0.72463787
    0.04059374]]

 [[ 9.304609    0.24425282  0.5528246  ...  0.0826336   0.21650867
    0.11110298]]

 [[ 3.9160197   0.19689088  0.3973506  ...  0.06380919  0.15808202
    0.03418554]]

 ...

 [[ 2.411285    0.54745793  3.17768    ...  1.273254    4.4564133
    3.2533772 ]]

 [[10.249314    0.41098285  1.1365355  ...  0.14434719  0.95457923
    0.01472451]]

 [[ 0.23575357  0.0997358   0.27453765 ...  0.36648452  0.9224097
    0.06749362]]]


100%|██████████| 219/219 [00:20<00:00, 10.82it/s]


In [37]:
train_data = np.squeeze(train_data)
test_data = np.squeeze(test_data)

In [58]:
elements = np.random.choice(len(train_data), int(0.3*len(train_data)), False)
explainer = shap.KernelExplainer(predict, train_data[elements])
shap_values_test = explainer.shap_values(test_data, nsamples=30, l1_reg='bic')

Using 196 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
  6%|▋         | 14/219 [00:17<04:30,  1.32s/it]Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=2.725e-03, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=3.842e-04, with an active set of 8 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=2.982e-04, with an active set of 8 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 16 it

 25%|██▌       | 55/219 [01:15<03:54,  1.43s/it]Regressors in active set degenerate. Dropping a regressor, after 16 iterations, i.e. alpha=8.435e-06, with an active set of 12 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=5.726e-03, with an active set of 3 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 7 iterations, i.e. alpha=2.797e-03, with an active set of 7 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 14 iterations, i.e. alpha=1.378e-04, with an active set of 12 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps paramete

 50%|████▉     | 109/219 [02:34<02:39,  1.45s/it]Regressors in active set degenerate. Dropping a regressor, after 13 iterations, i.e. alpha=1.165e-05, with an active set of 11 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 14 iterations, i.e. alpha=5.625e-06, with an active set of 12 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 14 iterations, i.e. alpha=4.560e-06, with an active set of 12 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 14 iterations, i.e. alpha=1.307e-06, with an active set of 12 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps par

 61%|██████    | 134/219 [03:11<02:06,  1.48s/it]Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=2.380e-03, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 10 iterations, i.e. alpha=4.399e-04, with an active set of 10 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 10 iterations, i.e. alpha=3.532e-04, with an active set of 10 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=1.201e-04, with an active set of 12 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps param

 87%|████████▋ | 190/219 [04:37<00:44,  1.54s/it]Regressors in active set degenerate. Dropping a regressor, after 6 iterations, i.e. alpha=1.656e-04, with an active set of 6 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 7 iterations, i.e. alpha=8.051e-05, with an active set of 7 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Regressors in active set degenerate. Dropping a regressor, after 13 iterations, i.e. alpha=2.221e-05, with an active set of 11 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 14 iterations, alpha=2.796e-05, previous alpha=1.961e-05, with an active set of 11 regressors.
Regressors in active se

100%|██████████| 219/219 [05:19<00:00,  1.46s/it]


In [63]:
test_data[0]

array([4.575468  , 0.83224285, 1.2297149 , 1.7188413 , 1.2518927 ,
       1.1866384 , 0.40216124, 0.6808334 , 0.21566421, 0.24898162,
       0.26711395, 0.3909327 , 0.72463787, 0.        ], dtype=float32)

In [59]:
print(shap_values_test[0].shape)

(219, 14)


In [66]:
import sys
sys.path.append("../architectural_style_classification")
from utils.knowledge_graph import GED_metric
d = GED_metric(test_data, shap_values_test, dataset='MonumenAI')

In [67]:
d

2.634703196347032